# Figure 2
Here, Figure 2 is created.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import seaborn
import sklearn.decomposition
import scipy.linalg
import pandas as pd
%matplotlib inline

In [ ]:
dataFilename = "../generatedData/fig2.h5"

In [ ]:
seaborn.set_style("white")

In [ ]:
signal = np.array([[0.2, 0.6, 0.0, -0.7, -0.2], [0.0, -0.3, 0.2, 0.0, 0.5]]).T

In [ ]:
def binSpikes(spikes):
    assert(isinstance(spikes, pd.DataFrame))
    spikes["timeBin"] = np.floor(spikes.time / 5e-2).astype(np.int)
    binned = spikes.groupby(["timeBin", "neuron"]).size().unstack(fill_value=0)
    return binned.reindex_axis(np.arange(1000), 1, fill_value=0)

In [ ]:
def make_plot(x, spikes, K, Phi, signal, stepsPerStep=5000, fixPerStep=1000, filename=None):
    binned = binSpikes(spikes)
    pca = sklearn.decomposition.PCA().fit(binned.values)
    fa = sklearn.decomposition.FactorAnalysis(n_components=2).fit(binned.values)
    zPca = np.copy(pca.components_[:2,:]).T
    zPca[:,1] *= -1
    zFa = -np.copy(fa.components_).T
    #zFa[:, 0] *= -1
    #correlationDf = pd.DataFrame({
    #    '$\Phi^T$': phi.values.flatten(),
    #    '$Z_\mathrm{PCA}$': zPca.flatten(),
    #    '$Z_\mathrm{FA}$': zFa.flatten()
    #})
    #correlationDf = correlationDf[['$K$', '$\Phi^T$', '$Z_\mathrm{PCA}$', '$Z_\mathrm{FA}$']]
    
    traces = x.values
    for i in range(len(traces)):
        if i%stepsPerStep < fixPerStep:
            traces[i] = signal[i//stepsPerStep]
    
    seaborn.reset_defaults()
    seaborn.set_context(context="paper")
    seaborn.set_style('white')
    
    fig, axs = plt.subplots(1,3,figsize=(6.5,1.5), gridspec_kw={'width_ratios': [0.6,0.02,0.3], 'wspace': .5},dpi=300)
    #corrAx = fig.add_axes([0, 0.3, 1, 0.7])
    cbarAx = fig.add_axes([0.84, 0.8, 0.1, 0.05])
    order = (K.values, phi.values, zPca, zFa)
    angles = np.array([[scipy.linalg.subspace_angles(a, b).mean() for a in order] for b in order])
    labels = ("$K$", "$\Phi^T$", "$Z_{PCA}$", "$Z_{FA}$")
    seaborn.heatmap(np.cos(angles), vmin=0, vmax=1, annot=True, square=True, fmt=".2f",
                    cmap='inferno', cbar_kws={ 'orientation': 'horizontal', 'ticks': [0,0.5,1.0]},
                    mask=1-np.tri(4), ax=axs[2], cbar_ax=cbarAx, annot_kws={'fontsize': 8},#, 'color': 'w'},
                    xticklabels=labels, yticklabels=labels
                   )
    evr = pca.explained_variance_ratio_[:50]
    axs[1].bar(np.zeros(50)+.5, evr, 1, [0]+list(np.cumsum(evr)[:-1]), color=seaborn.color_palette(), edgecolor="none")
    axs[1].set_ylim(0,1)
    axs[1].set_xlim(0,1)
    axs[1].set_xticks([])
    axs[1].set_yticks(np.linspace(0,1,5))
    axs[1].set_yticklabels(["%.0f%%" % t for t in np.linspace(0,100,5)])
    #
    #axs[1].set_ylabel("Fraction of variance")
    seaborn.despine(left = True, bottom=True, trim=False, fig=fig)
    
    selectedSpikes = spikes[spikes.neuron <= 50]
    axs[0].plot(selectedSpikes.time, (selectedSpikes.neuron/25.0 - 1.0), '.', ms=2, color='darkgray', markeredgewidth=0.0)
    axs[0].plot(np.linspace(0,2.5,stepsPerStep*len(signal)), traces[:,0])
    axs[0].plot(np.linspace(0,2.5,stepsPerStep*len(signal)), traces[:,1])
    #axs[0].set_ylim(-250, 250)
    axs[0].set_xlabel("Time [s]")
    axs[0].set_xlim(0,2.5)
    #seaborn.despine(trim=True)
    for i in range(5):
        axs[0].axvspan(i*0.5,i*0.5+0.1, alpha=.1)
    #plt.tight_layout()
    if filename is None:
        plt.show()
    else:
        plt.savefig(filename, dpi=300, bbox_inches="tight")

In [ ]:
x = pd.read_hdf(dataFilename, "/nef/original/x")
spikes = pd.read_hdf(dataFilename, "/nef/original/spikes")
K = pd.read_hdf(dataFilename, "/nef/original/K")
phi = pd.read_hdf(dataFilename, "/nef/original/phi")
make_plot(x, spikes, K, phi, signal, stepsPerStep=500, fixPerStep=100, filename='nefFig2.svg')

In [ ]:
x = pd.read_hdf(dataFilename, "/force/original/x")[950000:]
spikes = pd.read_hdf(dataFilename, "/force/original/spikes")
spikes = spikes[spikes.time>=47.5]
spikes.neuron -= 1
spikes.time -= 47.5
K = pd.read_hdf(dataFilename, "/force/original/K")
phi = pd.read_hdf(dataFilename, "/force/original/phi")
make_plot(x, spikes, K, phi, signal, stepsPerStep=10000, fixPerStep=2000, filename='forceFig2.svg')

In [ ]:
x = pd.read_hdf(dataFilename, "/ec/original/x") / 100.0
spikes = pd.read_hdf(dataFilename, "/ec/original/spikes")
spikes.neuron -= 1
K = pd.read_hdf(dataFilename, "/ec/original/K")
phi = pd.read_hdf(dataFilename, "/ec/original/phi")
make_plot(x, spikes, K, phi, signal, filename='ecFig2.svg')